https://www.kaggle.com/slm37102/ranzcr-clip-fastai-starter/

In [ ]:
from fastai.vision.all import *

In [ ]:
path = Path('../input/ranzcr-clip-catheter-line-classification')

In [ ]:
train_df = pd.read_csv(path/'train.csv')
train_df

In [ ]:
columns = list(train_df.columns[1:12])

In [ ]:
def get_x(r):
    return path/'train'/(r['StudyInstanceUID']+'.jpg')

def get_y(r):
    return r[columns].values.tolist()

def get_data(size=224, bs=128, data_df=train_df):
    dblock = DataBlock(blocks=(ImageBlock, MultiCategoryBlock(encoded=True, vocab=columns)),
                       splitter=RandomSplitter(seed=42),
                       get_x = get_x,
                       get_y = get_y,
                       item_tfms = RandomResizedCrop(460, min_scale=0.75, ratio=(1.,1.)),
                       batch_tfms = [*aug_transforms(size=size, flip_vert=True, max_warp=0),
                                     Normalize.from_stats(*imagenet_stats)])
    return dblock.dataloaders(data_df, bs=bs)


In [ ]:
dls = get_data()
dls.show_batch()

In [ ]:
learn = cnn_learner(dls, resnet50, metrics=accuracy_multi, cbs=MixUp).to_native_fp16()

In [ ]:
learn.fine_tune(5)

In [ ]:
learn = learn.to_native_fp32()

In [ ]:
submission_df = pd.read_csv(path/'sample_submission.csv')
submission_df.iloc[:,1:] = submission_df.iloc[:,1:].astype(float)
submission_df

In [ ]:
test_data_path = submission_df['StudyInstanceUID'].apply(lambda x: path/'test'/(x + '.jpg'))
tst_dl = learn.dls.test_dl(test_data_path)1
preds, targs = lean.tta(dl= tst_dl)

In [ ]:
submission_df[columns] =pd.DataFrame(preds, columns= columns)
submission_df

In [ ]:
submission_df.to_csv('submission.csv', index=False)